In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import seaborn as sns
rc('font', family = 'AppleGothic')
from sklearn.linear_model import LinearRegression

In [2]:
train = pd.read_csv("train.csv")
data = pd.read_csv('external_data.csv',header='infer')

In [3]:
train.shape

(2952, 15)

## 0. 전처리

- 세대수 = 총세대수
- 기본임대보증금 = 임대보증금
- 주택유형명 = 임대건물구분
- 공급유형명 = 공급유형
- 광역도시명 = 지역
- 공급전용면적 = 전용면적
- 주차수 = 단지내주차면수(완전히 동일하지는 않음!!)

In [4]:
train['임대료'] = train['임대료'].replace('-',np.nan)
train['임대보증금'] = train['임대보증금'].replace('-',np.nan)
train['임대료'] = train['임대료'].apply(lambda x : float(x))
train['임대보증금'] = train['임대보증금'].apply(lambda x : float(x))

In [5]:
data = data.rename(columns = {'세대수':'총세대수','기본임대보증금':'임대보증금','주택유형명':'임대건물구분','공급유형명':'공급유형','광역시도명':'지역','공급전용면적':'전용면적','주차수':'단지내주차면수'})

In [6]:
print(sorted(train['지역'].unique()))
print(sorted(data['지역'].unique()))

['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']
['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [7]:
print(sorted(data['공급유형'].unique()))
print(sorted(train['공급유형'].unique()))

['10년임대', '50년임대', '5년임대', '국민임대', '매입임대', '영구임대', '장기전세', '행복주택']
['공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)', '국민임대', '영구임대', '임대상가', '장기전세', '행복주택']


In [8]:
data.loc[data.공급유형.isin(['10년임대']), '공급유형'] = '공공임대(10년)'
data.loc[data.공급유형.isin(['50년임대']), '공급유형'] = '공공임대(50년)'
data.loc[data.공급유형.isin(['5년임대']), '공급유형'] = '공공임대(5년)'

In [9]:
data1 = data[data['임대건물구분']=='아파트']
train1 = train[train['임대건물구분']=='아파트']

In [10]:
data1 = data1[['지역','총세대수','전용면적','단지명','공급유형','임대건물구분','임대보증금','단지내주차면수','기관명','도로명주소']]
train1 = train1[['단지코드','총세대수','지역','전용면적','임대보증금','공급유형','임대건물구분','임대보증금','단지내주차면수']]

## 기준1 : 지역, 공급유형, 총세대수

In [11]:
print('전체 단지코드: ',train['단지코드'].nunique())

전체 단지코드:  423


In [12]:
result1 = pd.merge(train1,data1,how='left',on=['지역','공급유형','총세대수'])[['단지코드','단지명']].drop_duplicates()

In [13]:
print('NA 개수: ',result1['단지명'].isna().sum())
print('특정된 단지코드 수: ',train['단지코드'].nunique()-result1['단지명'].isna().sum())
print('전체 단지코드 수: ',train['단지코드'].nunique())
print('NA 비율: ', round(result1['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  94
특정된 단지코드 수:  329
전체 단지코드 수:  423
NA 비율:  0.22


## 기준 2 : 지역, 공급유형, 전용면적, 임대보증금

In [14]:
datana = result1[result1['단지명'].isna()==True]['단지코드'].values
n = len(datana)

In [15]:
datana = train.loc[train['단지코드'].isin(datana)]
datana = datana[datana['임대건물구분']=='아파트']

In [16]:
result2 = pd.merge(datana,data1,how='left',on=['지역','공급유형','전용면적','임대보증금'])[['단지코드','단지명']].drop_duplicates()

In [17]:
print('NA 개수: ',result2['단지명'].isna().sum())
print('특정된 단지코드 수: ',n-result2['단지명'].isna().sum())
print('전체 단지코드 수: ',n)
print('NA 비율: ', round(result2['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  39
특정된 단지코드 수:  55
전체 단지코드 수:  94
NA 비율:  0.09


## 기준3 : 지역, 공급유형, 전용면적, 단지내주차면수

In [18]:
datana = result2[result2['단지명'].isna()==True]['단지코드'].values
n = len(datana)

In [19]:
datana = train.loc[train['단지코드'].isin(datana)]
datana = datana[datana['임대건물구분']=='아파트']

In [20]:
result3 = pd.merge(datana,data1,how='left',on=['지역','공급유형','전용면적','단지내주차면수'])[['단지코드','단지명']].drop_duplicates()

In [21]:
print('NA 개수: ',result3['단지명'].isna().sum())
print('특정된 단지코드 수: ',n-result3['단지명'].isna().sum())
print('전체 단지코드 수: ',n)
print('NA 비율: ', round(result3['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  28
특정된 단지코드 수:  11
전체 단지코드 수:  39
NA 비율:  0.07


In [22]:
result = result1.dropna()
result = result.append(result2.dropna())
result= result.append(result3.dropna()).reset_index(drop=True)

In [23]:
na_value=[]
for i in train['단지코드'].unique():
    if i not in result['단지코드'].unique():
        na_value.append(i)

In [24]:
na_value

['C1899', 'C2245', 'C1584', 'C1156', 'C2020']

- 아래 데이터는 하나의 단지코드 별로 두개 이상의 단지명 보유 $\rightarrow$ 잘못 merge된 케이스

In [25]:
#전체
tmp = pd.DataFrame(result.groupby('단지코드').count()['단지명']).reset_index()
tmp = np.array(tmp[tmp['단지명']>=2]['단지코드'])
print(tmp)
print(len(tmp))

['C1022' 'C1039' 'C1068' 'C1085' 'C1109' 'C1175' 'C1206' 'C1207' 'C1234'
 'C1258' 'C1269' 'C1307' 'C1341' 'C1344' 'C1439' 'C1451' 'C1601' 'C1616'
 'C1681' 'C1732' 'C1740' 'C1744' 'C1790' 'C1859' 'C1874' 'C1875' 'C1894'
 'C1965' 'C1970' 'C2034' 'C2038' 'C2070' 'C2082' 'C2085' 'C2109' 'C2127'
 'C2132' 'C2133' 'C2135' 'C2173' 'C2186' 'C2190' 'C2227' 'C2232' 'C2258'
 'C2289' 'C2310' 'C2325' 'C2352' 'C2378' 'C2393' 'C2416' 'C2453' 'C2530'
 'C2563' 'C2597' 'C2621' 'C2627' 'C2635' 'C2657']
60


## 상가 drop

In [26]:
drop = [460, 462, 465, 469, 470, 411, 414, 415, 416, 417, 418, 419, 420, 421, 423, 424 ,425, 426, 427, 428,432,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,456,458,459,363, 365,366,370,371,372,373,375,378,379,380,381,382,385,386,388,389,390,391,393,394,395,396,391,393,394,395,396,406]

In [27]:
result = result.drop(drop,axis=0).reset_index(drop=True)

In [28]:
result.to_csv("result.csv",index=False) #상가 부분 제거!!

## 아파트 drop

In [29]:
tmp = pd.DataFrame(result.groupby('단지코드').count()['단지명']).reset_index()
tmp = np.array(tmp[tmp['단지명']>=2]['단지코드'])
print(tmp)
print(len(tmp))

['C1022' 'C1039' 'C1068' 'C1085' 'C1175' 'C1207' 'C1234' 'C1258' 'C1269'
 'C1307' 'C1341' 'C1344' 'C1439' 'C1451' 'C1601' 'C1616' 'C1681' 'C1732'
 'C1740' 'C1744' 'C1859' 'C1894' 'C1965' 'C1970' 'C2070' 'C2082' 'C2085'
 'C2127' 'C2133' 'C2173' 'C2186' 'C2227' 'C2232' 'C2325' 'C2352' 'C2378'
 'C2393' 'C2453' 'C2530' 'C2563' 'C2597' 'C2627' 'C2635' 'C2657']
44


In [30]:
# 'C1681', 'C1740', 'C1744', 'C1790', 'C1859', 'C1874', 'C1875', 'C1894', 'C1965', 'C1970', 'C2034', 'C2038', 'C2070'
# 'C2416', 'C2453', 'C2530','C2563'

In [31]:
# 'C2416'
# 'C2453' 성남여수1 
# 'C2530' 원주개운LH2단지아파트
# 'C2563' 안산팔곡주공아파트
# [329,171,136]

In [32]:
ind = [116,118,415,85,87,88,375,376,377,378,379,456,221,452,156,329,171,136,257, 250, 71, 123, 134, 67, 390, 392, 393, 92, 93, 164,149, # 'C1022'
       150, # 'C1068'
       385, # 'C1085'
       185, # 'C1085'
       32, # 'C1175'
       354, # 'C1207'
       53, # 'C1234'
       153, # 'C1269'
       182, # 'C1307'
       395,396, # 'C1439'
       139,140, # 'C1601'
       370, # 'C1616'
       191, # 'C2325'
       266,409, # 'C2352'
       224]

In [33]:
result = result.drop(ind,axis=0)
result = result.drop_duplicates().reset_index(drop=True)

In [34]:
result.to_csv("result0708.csv",index=False) 

In [35]:
price = pd.read_csv("store_tr0708.csv") 

In [36]:
price['임대료'] = price['임대료'].replace(0,np.nan)
price['임대보증금'] = price['임대보증금'].replace(0,np.nan)

In [37]:
price = price[['단지코드','전용면적','임대료','임대보증금','단지명']]

In [38]:
store_price = pd.merge(price,result,on='단지코드',how='left').iloc[:,:-1].rename(columns={'단지명_x':'단지명'})
store_price = store_price.drop_duplicates()

In [39]:
# 상가 
tmp1 = train[train['임대건물구분']=='상가'] 
tmp1 = tmp1.drop(['임대료','임대보증금'],axis=1)
tmp1 = pd.merge(tmp1,store_price[['단지코드','전용면적','임대료','임대보증금','단지명']],how='left',on=['단지코드','전용면적']) #상가 임대료

In [40]:
print(train[(train['임대건물구분']=='아파트')].shape)
print(train[(train['임대건물구분']=='상가')].shape)

(2390, 15)
(562, 15)


In [41]:
# 아파트
tmp2 = train[train['임대건물구분']=='아파트']
tmp2 = pd.merge(tmp2,result[['단지코드','단지명']],how='left',on=['단지코드']) 

In [42]:
train = pd.concat([tmp2,tmp1])
train = train.reset_index(drop=True)

In [43]:
train[(train['임대건물구분']=='아파트')&(train['임대료'].isna()==True)]['단지코드'].unique()

array(['C1397', 'C1039', 'C1350', 'C1326', 'C1786', 'C2186'], dtype=object)

In [44]:
def get_area(code):
    area = train[(train['단지코드']==code)]['전용면적'].unique()
    ind = train[(train['단지코드']==code)].index
    print('단지명: ',train[train['단지코드']==code]['단지명'].unique())
    print('지역: ',train[train['단지코드']==code]['지역'].unique())
    print('면적 개수: ',train[train['단지코드']==code]['지역'].shape[0])
    for i,j in zip(area,ind):
        print(i,j,'/',end=' ')

In [45]:
def func(num,a,b):
    train.loc[num,'임대보증금'] = a
    train.loc[num,'임대료'] = b

In [46]:
def linear_func(code):
    tmp = train[(train['단지코드']==code)][['전용면적','임대료','임대보증금']]
    tmp = tmp.replace(0,np.nan)
    ind = tmp[tmp['임대료'].isna()==True].index
    train1 = tmp.dropna()
    train_X = np.array(train1['전용면적']).reshape(-1,1)
    y1 = np.array(train1['임대료'])
    y2 = np.array(train1['임대보증금'])
    test = tmp[tmp['임대료'].isna()==True]
    test_X = np.array(test['전용면적']).reshape(-1,1)
    lg = LinearRegression()
    lg1 = LinearRegression()
    lg.fit(train_X,y1) #임대료
    lg1.fit(train_X,y2) #임대보증금
    pred = lg.predict(test_X)
    pred1 = lg1.predict(test_X)
    train.loc[ind,'임대료'] = pred
    train.loc[ind,'임대보증금'] = pred1

## 아파트 보증금 NA imputation

In [47]:
#임대료가 없는 경우 
train.loc[1538:1540,'임대료']=0 #C1039
train.loc[1481:1486,'임대료']=0 #C1397

In [48]:
get_area('C1350') # 없음..
linear_func('C1350') 

단지명:  ['관저 LH다온숲 3단지']
지역:  ['대전광역시']
면적 개수:  13
74.94 1769 / 84.94 1770 / 84.96 1771 / 84.97 1772 / 51.99 1773 / 59.91 1774 / 59.92 1775 / 

In [49]:
get_area( 'C1326')
func(1985,7000000,135000)
func(1986,7000000,135000)
func(1987,7600000,142000)
func(1988,14800000,198000)
func(1989,23100000,259000)

단지명:  ['부산정관 7단지(A-1BL)']
지역:  ['부산광역시']
면적 개수:  5
24.72 1985 / 24.79 1986 / 26.83 1987 / 37.7 1988 / 46.94 1989 / 

In [50]:
get_area('C1786')
func(2118,13181000,62610)
func(2119,19397000,92130)
func(2120,30464000,144700)
func(2121,30464000,144700)

단지명:  ['춘천거두2 행복주택']
지역:  ['강원도']
면적 개수:  5
16.91 2118 / 26.9 2119 / 36.91 2120 / 

In [51]:
get_area('C2186') #없음
linear_func('C2186') 

단지명:  ['대구연경 LH천년나무 2단지']
지역:  ['대구광역시']
면적 개수:  6
29.17 2344 / 29.34 2345 / 37.43 2346 / 46.51 2347 / 26.37 2348 / 26.54 2349 / 

## NA 추가 Imputation
- C1109 C1439

In [52]:
tmp = train[(train['단지코드']=='C1109')&(train['전용면적']<100)][['전용면적','임대료','임대보증금']]
ind = tmp[tmp['임대료'].isna()==True].index
train_X = np.array([37.41,37.26,54.51,37.95,39.33]).reshape(-1,1)
y1 = np.array([220200,324000,256000,179000,360000])
y2 = np.array([13570000,7776000,6144000,4296000,8640000])
test = tmp[tmp['임대료'].isna()==True]
test_X = np.array(test['전용면적']).reshape(-1,1)
lg = LinearRegression()
lg1 = LinearRegression()
lg.fit(train_X,y1) #임대료
lg1.fit(train_X,y2) #임대보증금
pred = lg.predict(test_X)
pred1 = lg1.predict(test_X)
train.loc[ind,'임대료'] = pred
train.loc[ind,'임대보증금'] = pred1

## 특정 안 된 NA

In [53]:
def danji_imputation(code,danji):
    ind = train[train['단지코드']==code].index
    train.loc[ind,'단지명'] = danji

In [54]:
dic = {'C1109': '부산금곡주공4단지',
 'C2132': '모라주공3단지',
 'C1439': '주공아파트',
 'C1899': '사천벌리1주공',
 'C2644': '석림주공3단지',
 'C1206': '명륜주공2차아파트',
 'C2245': '진주문산 LH아파트',
 'C1584': '남양주별내 A1-2 행복주택리츠',
 'C1156': '청주동남 LH행복주택',
 'C2020': '부산용호(행복주택)'}

In [55]:
for i in dic:
    code = i
    danji = dic[i]
    danji_imputation(code,danji)

## train, test 주소 겹치는 거 하나로 특정 , 상가 na impuataion 다시
- {'모라주공아파트1단지': 'C2132', 
 '속초청초주공아파트': 'C1206', 
 '휴먼시아': 'C1379', 
 '송정주공아파트': 'C1569', # 도로명주소만 변경: 경상남도 창원시 의창구 동읍 용정길 26
 '광주첨단H-2': 'C1970'}

In [56]:
external = pd.read_csv("external_data.csv")
external = external[['단지명','도로명주소']].drop_duplicates()
external = external.groupby('단지명')['도로명주소'].first().reset_index() #첫번째 도로명주소 사용

In [57]:
train = pd.merge(train,external,how='left',on='단지명') #도로명주소 merge

In [58]:
train.loc[train['단지코드'].isin(['C2132']),'도로명주소'] = external[external['단지명']=='모라주공아파트1단지']['도로명주소'].values
train.loc[train['단지코드'].isin(['C1206']),'도로명주소'] = external[external['단지명']=='속초청초주공아파트']['도로명주소'].values
train.loc[train['단지코드'].isin(['C1379']),'도로명주소'] = external[external['단지명']=='휴먼시아']['도로명주소'].values
train.loc[train['단지코드'].isin(['C1569']),'도로명주소'] = '경상남도 창원시 의창구 동읍 용정길 26'
train.loc[train['단지코드'].isin(['C1970']),'도로명주소'] = external[external['단지명']=='광주첨단H-2']['도로명주소'].values

In [59]:
train.loc[train['단지코드'].isin(['C2132']),'단지명'] = '모라주공아파트1단지' 
train.loc[train['단지코드'].isin(['C1206']),'단지명'] = '속초청초주공아파트'
train.loc[train['단지코드'].isin(['C1379']),'단지명'] = '휴먼시아'
train.loc[train['단지코드'].isin(['C1970']),'단지명'] = '광주첨단H-2'

In [60]:
# 1206은 상가건물이어서 임대보증금 na 다시 채워줌 
# 6120000, 2636000,1224000
train.loc[2818:2821,'임대보증금'] = 13180000
train.loc[2818:2821,'임대료'] = 2636000
train.loc[2822:2825,'임대보증금'] = 6120000
train.loc[2822:2825,'임대료'] = 1224000
train.loc[2826,'임대보증금'] = 6120000*2
train.loc[2826,'임대료'] = 1224000 * 2

In [61]:
# 지하 상가 na 처리
train.loc[2827,'임대료'] = np.nan
train.loc[2827,'임대보증금'] = np.nan

## 지하 Imputation

In [62]:
tmp = train[train['전용면적']>100][['전용면적','임대료','임대보증금']]
ind = tmp[tmp['임대료'].isna()==True].index
train1 = tmp.dropna()
train_X = np.array(train1['전용면적']).reshape(-1,1)
y1 = np.array(train1['임대료'])
y2 = np.array(train1['임대보증금'])
test = tmp[tmp['임대료'].isna()==True]
test_X = np.array(test['전용면적']).reshape(-1,1)
base = LinearRegression()
base1 = LinearRegression()
base.fit(train_X,y1) #임대료
base1.fit(train_X,y2) #임대보증금
pred = base.predict(test_X)
pred1 = base1.predict(test_X)
train.loc[ind,'임대료'] = pred
train.loc[ind,'임대보증금'] = pred1

In [63]:
#c1439 concat
c1439 = pd.read_csv("c1439.csv")
c1439 = c1439[['단지코드','임대료','임대보증금','전용면적']]

In [64]:
ind = train[(train['단지코드']=='C1439')&(train['임대료'].isna()==True)].index

In [65]:
for i in ind:
    area = train.loc[i,'전용면적']
    train.loc[i,'임대료'] = c1439[c1439['전용면적']==area]['임대료'].values
    train.loc[i,'임대보증금'] = c1439[c1439['전용면적']==area]['임대보증금'].values

In [66]:
#도로명주소 없는 case: C1899
ind = train[train['도로명주소'].isna()==True].index
train.loc[ind,'도로명주소'] = '경상남도 사천시 주공1길 50'

In [67]:
train = train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철역','도보 10분거리 내 버스정류장 수':'버스정류장'})

## 버스정류장,지하철역

In [68]:
#버스정류장 
print(train[train['지역']=='경상남도']['버스정류장'].mean())
ind = train[train['버스정류장'].isna()==True].index
train.loc[ind,'버스정류장']=4

3.824657534246575


In [69]:
train[train['지하철역'].isna()==True]['지역'].unique()

for i in train[train['지하철역'].isna()==True]['지역'].unique():
    print(i,train[(train['지역']==i)]['지하철역'].unique())

ind = train[(train['지역']=='충청남도')&(train['지하철역'].isna()==True)].index
train.loc[ind,'지하철역'] = 0

ind = train[(train['지역']=='경상남도')&(train['지하철역'].isna()==True)].index
train.loc[ind,'지하철역'] = 0

train[train['지역']=='대전광역시'].corr()['지하철역']

X_train = train[(train['지역']=='대전광역시')&(train['지하철역'].isna()==False)]['버스정류장'].values.reshape(-1,1)
y_train = train[(train['지역']=='대전광역시')&(train['지하철역'].isna()==False)]['지하철역'].values.ravel()
X_test = train[(train['지역']=='대전광역시')&(train['지하철역'].isna()==True)]['버스정류장'].values.reshape(-1,1)

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(X_train, y_train)
pred = reg.predict(X_test)

ind = train[(train['지역']=='대전광역시')&(train['지하철역'].isna()==True)].index
train.loc[ind,'지하철역'] = pred

충청남도 [nan  0.]
대전광역시 [ 1.  0. nan]
경상남도 [ 0. nan]


In [70]:
train.loc[86,'도로명주소'] = '강원도 춘천시 공지로 234-16'
train.loc[87,'도로명주소'] = '강원도 춘천시 공지로 234-16'
train.loc[86,'단지명'] = '춘천효자8 단지'
train.loc[87,'단지명'] = '춘천효자8 단지'

- "C1649","C2431"
- "C1397","C2085"

In [71]:
ind = train[train['단지코드']=='C2431'].index 
train.loc[ind,'단지명'] = '양산신도시LH아파트5단지'
train.loc[ind,'단지코드'] = 'C1649'

In [72]:
ind = train[train['단지코드']=='C1397'].index 
train.loc[ind,'단지명'] = '강남 A5BL (LH강남힐스테이트)'
train.loc[ind,'단지코드'] = 'C2085'

- C1925 오타 발견

In [73]:
result[result['단지코드']=='C1925']

,단지코드,단지명
330,C1925,입암주공3단지아파트


In [74]:
external[external['단지명']=='입암주공3단지아파트']

,단지명,도로명주소
4045,입암주공3단지아파트,강원도 강릉시 강변로 510


In [75]:
train.loc[train['단지코드'].isin(['C1925']),'단지명'] = '입암주공3단지아파트'
train.loc[train['단지코드'].isin(['C1925']),'도로명주소'] = '강원도 강릉시 강변로 510'

- 대전둔산 3 도로명주소 수정

In [76]:
train.loc[train['단지명'].isin(['대전둔산3']),'도로명주소'] = '대전광역시 서구 월평북로 13'

In [77]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
train.to_csv(os.path.join(path, "train_processed.csv"),index=False)

## test data

In [78]:
test = pd.read_excel("test_imputed.xlsx")

In [79]:
test['임대보증금'] = test['임대보증금'].replace(0,np.nan)
test['임대료'] = test['임대료'].replace(0,np.nan)

In [80]:
tmp = test[test['전용면적']>100][['전용면적','임대료','임대보증금']]
ind = tmp[tmp['임대료'].isna()==True].index
test1 = tmp.dropna()
train_X = np.array(train[train['전용면적']>100]['전용면적']).reshape(-1,1)
y1 = np.array(train[train['전용면적']>100]['임대료'])
y2 = np.array(train[train['전용면적']>100]['임대보증금'])
test_ = tmp[tmp['임대료'].isna()==True]
test_X = np.array(test_['전용면적']).reshape(-1,1)
base = LinearRegression()
base1 = LinearRegression()
base.fit(train_X,y1) #임대료
base1.fit(train_X,y2) #임대보증금
pred = base.predict(test_X)
pred1 = base1.predict(test_X)
test.loc[ind,'임대료'] = pred
test.loc[ind,'임대보증금'] = pred1

In [81]:
test.loc[test.단지코드=='C2675','단지명'] = '흥덕마을휴먼시아1단지아파트(110동~115동)'

In [82]:
test = test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철역','도보 10분거리 내 버스정류장 수':'버스정류장'})

In [83]:
#자격유형
test.loc[(test.단지코드=='C2411')&(test.자격유형.isna()), '자격유형'] = 'A'
test.loc[(test.단지코드=='C2253')&(test.자격유형.isna()), '자격유형'] = 'C'

In [84]:
#지하철역
ind = test[(test['지역']=='충청남도')&(test['지하철역'].isna()==True)].index
test.loc[ind,'지하철역'] = 0

X_test = test[(test['지역']=='대전광역시')&(test['지하철역'].isna()==True)]['버스정류장'].values.reshape(-1,1)
pred = reg.predict(X_test)

ind = test[(test['지역']=='대전광역시')&(test['지하철역'].isna()==True)].index
test.loc[ind,'지하철역'] = pred

In [85]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
test.to_csv(os.path.join(path, "test_processed.csv"),index=False)